# LlamaIndexでRAG

## 目次
- [概要](#概要)
- [参考](#参考)
- [チェック](#チェック)
- [エージェントにRAGを追加する](#エージェントにRAGを追加する)

## 概要
- LlamaIndex（公式）をトレースして基本的な利用方法を確認する。
- 破壊的に変更が発生するまで使えるでしょう。
- 破壊的に変更が発生後は、公式サイトの当該バージョンの情報（≒一次情報）をあたって。

## 参考

LLMのRAG - .NET 開発基盤部会 Wiki  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?LLM%E3%81%AERAG
- 知識情報の分割
- 知識情報の埋め込み
- 質問の入力（Query Input）
- 質問の埋め込み（Query Embedding）
- 情報の検索（Information Retrieval）
- 情報の生成（Information Generation）
- 回答の提供（Answer Delivery）

LlamaIndex - .NET 開発基盤部会 Wiki  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?LlamaIndex
- Loading
- Indexing
- Storing
- Querying
- Evaluation

## チェック

In [ ]:
#!pip list

In [ ]:
#%env

## 準備

### ライブラリ読み込み

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.core.agent import ReActAgent

## エージェント

### 最も簡単なエージェント

#### ライブラリ読み込み

In [ ]:
from llama_index.core.tools import FunctionTool

#### LLMの定義
エージェントでは揺らぐと困るので、temperature=0 に設定している。

In [ ]:
# ollama
llm = Ollama(model="Llama3", temperature=0, request_timeout=360.0)

#### ファンクションツールのエージェント

##### ファンクションツールの定義

In [ ]:
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b

multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

##### エージェントの構成

In [ ]:
agent = ReActAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)

##### エージェントの実行

In [ ]:
response = agent.chat("What is 20 + (2 * 4) ? Use a tool to calculate every step.")
print(response)

### エージェントにRAGを追加する

#### 準備

##### 使用する変数

In [ ]:
DATA_DIR = "./llamaindex/data/2023_canadian_budget"
PERSIST_DIR = "./llamaindex/storage/2023_canadian_budget"
CHROMA_DIR = "./llamaindex/chroma_db/2023_canadian_budget"

##### ライブラリ読み込み

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

#### 簡単なテスト（永続化付き）

##### Settings

In [ ]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
Settings.llm = llm # 複数箇所で使うのでグローバルに設定

##### Indexing & Storing

In [ ]:
import os.path
from llama_index.core import (
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader(DATA_DIR).load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

##### Querying

In [ ]:
# Either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query(
    "What was the total amount of the 2023 Canadian federal budget?"
    "\n Please answer with one number."
)
print(response)

#### ツール作成

##### ファンクション・ツール

In [ ]:
from llama_index.core.tools import FunctionTool

In [ ]:
# function tools
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b

multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

##### クエリエンジンツール

In [ ]:
from llama_index.core.tools import QueryEngineTool

In [ ]:
budget_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="canadian_budget_2023",
    description="A RAG engine with some basic facts about the 2023 Canadian federal budget.")

#### エージェントにツールを実行させる

##### ライブラリ読み込み

In [ ]:
from llama_index.core.agent import ReActAgent

##### RAG込エージェント定義

In [ ]:
agent = ReActAgent.from_tools([multiply_tool, add_tool, budget_tool], verbose=True)

##### RAG込エージェント実行
- SLMだと上手く動作しない問題。
- そしてデバッグと対策も不明（笑）

In [ ]:
response = agent.chat("What is the total amount of the 2023 Canadian federal budget multiplied by 3?")
print(response)

#### 追加のインストレーション1

In [ ]:
# 不要（OpenAIは依存関係パッケージらしい）

#### 追加のライブラリ読み込み1

In [ ]:
# なし

#### ...

#### 追加のインストレーション2

```bash
!pip install llama-index-llms-ollama
!pip install llama-index-embeddings-huggingface
```

#### 追加のライブラリ読み込み2

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama

#### インデックス作成
最も基礎的で、オンメモリのセマンティック検索のインデックス。
- RAGで言うと：知識情報の分割、知識情報の埋め込み
- LlamaIndexで言うと：Loading、Indexing

##### Loading

In [ ]:
documents = SimpleDirectoryReader(DATA_DIR1).load_data()

##### Indexing
- 以下では、VectorStoreIndexを使用している。
- SummaryIndexやKnowledgeGraphIndexなどのindexもある。
- 質問内容によって最適なIndexが異なる可能性がある。

In [ ]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# ollama
Settings.llm = Ollama(model="Llama3", request_timeout=360.0)

index = VectorStoreIndex.from_documents(
    documents,
)

#### RAGのRetrieval部
- RAGで言うと：Query Input、Query Embedding、Information Retrieval
- LlamaIndexで言うと：Querying、Evaluation

##### Querying

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)

##### Evaluation

In [ ]:
# ...

#### ログの有効化
抽象化度が高いので、ログの有効化をしても良いが、結局、欲しい所が出きってない感もある。

```Python
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
```

#### 永続化して実行
前述のコードに、LlamaIndexで言うStoringの概念を追加したもの。
- 永続化は、Document Store、Vector Store、Index Storeに、Storage Contextを設定する。
- 既出の、Loadingの所で、Document Storeから読み出している。
- Indexingでは、Vector Store、Index Storeに書き出し（永続化し）す。

In [ ]:
import os.path
from llama_index.core import (
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader(DATA_DIR1).load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# Either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)

#### 使用する変数

In [ ]:
DATA_DIR2 = "./llamaindex/data/2023_canadian_budget"

#### ライブラリ読み込み

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama

In [ ]:
documents = SimpleDirectoryReader(DATA_DIR2).load_data()

# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# ollama
Settings.llm = Ollama(model="Llama3", temperature=0)

index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()
response = query_engine.query(
    "What was the total amount of the 2023 Canadian federal budget?"
)
print(response)